In [1]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
import glob
from sklearn.model_selection import train_test_split 
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, SGD
import cv2
from imblearn.over_sampling import SMOTE
import pywt

C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

# Label extraction

In [10]:
df=pd.read_csv("oasis_longitudinal_demographics.csv")

In [11]:
df.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [12]:
labels=df['Group']

In [13]:
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [14]:
label_to_id_dict

{'Converted': 0, 'Demented': 1, 'Nondemented': 2}

In [15]:
label_ids = np.array([label_to_id_dict[x] for x in labels])

In [16]:
label_ids

array([2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 1, 1,
       1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 0,
       0, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 0, 0,
       0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1,

# Image extraction

In [302]:
files=glob.glob("OAS2_RAW_PART1/*")

In [303]:
features=[]
for file_path in files:
    filename = os.path.join(file_path, 'RAW/mpr-1.nifti.img')
    img = nib.load(filename)
    image_data = img.get_fdata()
    image=ndimage.rotate(image_data[:, :, 64,0],90)
    ret, thresh3 = cv2.threshold(image, 120, 255, cv2.THRESH_TOZERO)
    c=pywt.wavedec2(thresh3[70:150,70:170],'db5',mode='periodization',level=2)
    #image = cv2.cvtColor(thresh3, cv2.COLOR_RGB2BGR)
    cA2=c[0]
    (cH1,cV1,cD1)=c[-1]
    (cH2,cV2,cD2)=c[-2]
    features.append(cD1)

In [304]:
len(features)

209

In [305]:
files=glob.glob("OAS2_RAW_PART2/*")

In [306]:
for file_path in files:
    filename = os.path.join(file_path, 'RAW/mpr-1.nifti.img')
    img = nib.load(filename)
    image_data = img.get_fdata()
    image=ndimage.rotate(image_data[:, :, 64,0],90)
    ret, thresh3 = cv2.threshold(image, 120, 255, cv2.THRESH_TOZERO)
    c=pywt.wavedec2(thresh3[70:150,70:170],'db5',mode='periodization',level=2)
    #image = cv2.cvtColor(thresh3, cv2.COLOR_RGB2BGR)
    cA2=c[0]
    (cH1,cV1,cD1)=c[-1]
    (cH2,cV2,cD2)=c[-2]
    features.append(cD1)

In [307]:
len(features)

373

In [308]:
features=np.array(features)

# Train test

In [309]:
X_train, X_test, y_train, y_test = train_test_split(features,label_ids, test_size = 0.20)  

In [310]:
X_train[0].shape

(40, 50)

In [311]:
#Normalize color values to between 0 and 1
X_train = X_train/255
X_test = X_test/255
y_train = tensorflow.keras.utils.to_categorical(y_train, 4)
y_test = tensorflow.keras.utils.to_categorical(y_test, 4)

#Make a flattened version for some of our models
#1st level feature extraction
#X_flat_train = X_train.reshape(X_train.shape[0], 128*128*1)
#X_flat_test = X_test.reshape(X_test.shape[0], 128*128*1)

#2nd level feature extraction 
#X_flat_train = X_train.reshape(X_train.shape[0], 64*64*1)
#X_flat_test = X_test.reshape(X_test.shape[0], 64*64*1)

#Augmented level 2 feature extraction 
#X_flat_train = X_train.reshape(X_train.shape[0], 10*10*5)
#X_flat_test = X_test.reshape(X_test.shape[0], 10*10*5)

#Augmented level 1 feature extraction 
X_flat_train = X_train.reshape(X_train.shape[0], 20*20*5)
X_flat_test = X_test.reshape(X_test.shape[0], 20*20*5)

In [312]:
#X_3d_train = X_train.reshape(X_train.shape[0], 64,64,1)
#X_3d_test = X_test.reshape(X_test.shape[0], 64,64,1)

In [313]:
#X_3d_train.shape

In [314]:
X_flat_train.shape

(298, 2000)

In [315]:
y_train=label_ids[:298]
y_test=label_ids[298:373]

# Multilayer model with oversampling

In [316]:
sm=SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(X_flat_train, y_train)

In [317]:
train_data.shape

(456, 2000)

## No oversampling

In [325]:
model_deep = Sequential()

model_deep.add(Dense(512, activation='relu', input_shape=(train_data.shape[1],)))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(256, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(128, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(128, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(64, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(64, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(3, activation='softmax'))

model_deep.summary()

model_deep.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history_deep = model_deep.fit(X_flat_train, y_train,
                          batch_size=32,
                          epochs=10,
                          verbose=1,
                          validation_data=(X_flat_test, 
                                           y_test))
score = model_deep.evaluate(X_flat_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_57"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_294 (Dense)            (None, 512)               1024512   
_________________________________________________________________
dropout_245 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_295 (Dense)            (None, 256)               131328    
_________________________________________________________________
dropout_246 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_296 (Dense)            (None, 128)               32896     
_________________________________________________________________
dropout_247 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_297 (Dense)            (None, 128)             

## Oversampling

In [319]:
model_deep = Sequential()

model_deep.add(Dense(256, activation='relu', input_shape=(train_data.shape[1],)))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(128, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(128, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(64, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(64, activation='relu'))
model_deep.add(Dropout(0.05))
model_deep.add(Dense(3, activation='softmax'))

model_deep.summary()

model_deep.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history_deep = model_deep.fit(train_data, train_labels,
                          batch_size=32,
                          epochs=10,
                          verbose=1,
                          validation_data=(X_flat_test, 
                                           y_test))
score = model_deep.evaluate(X_flat_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_263 (Dense)            (None, 256)               512256    
_________________________________________________________________
dropout_219 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_264 (Dense)            (None, 128)               32896     
_________________________________________________________________
dropout_220 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_265 (Dense)            (None, 128)               16512     
_________________________________________________________________
dropout_221 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_266 (Dense)            (None, 64)              